# Individual Assignment 3 - Querying data
In this assignment, students are going to focus on getting data out of a database, using a few different methods

## Goal
The goal of this assignment is to understand how to get data out of a database using SQL queries directly, pandas and also directly into our objects using SQLAlchemy 

## Deliverables
1. A PDF of this notebook OR a link to the repository where the final version of the notebook is store.

## Background
Relational databases are super helpful way to store data and settings for applications.  They are efficient at managing space, easy to maintain and provide an extra layer of security to the data. Being able to load and save data to a database from a program brings a great deal of flexibility to our ability to automate.

## Instructions
**BEFORE YOU BEGIN**, if you haven't forked/cloned this from github you will need to ensure that you also get the `chinook.db` and `laptopsales.db` file as well as the entire `images` folder and put these in the same directory as the notebook.  See below.  If you have forked/cloned this repository, everything you need is already in the right place.

```text
assignments
│   ├── Ind Assignment 3 - Querying Data.ipynb
│   ├── images
│   │   ├── a1-image-1.png
│   │   ├── a1-image-2.png
│   │   ├── a1-image.png
│   │   ├── a3-invoices.png
│   │   └── sqlite-sample-database-color.jpg
|   ├── chinook.db
│   └── laptopsales.db
```
For this assignment, you should work through this notebook.  Executing each of the cells and writing code in the various YOUR TURN cells in this notebook.

You have a choice of either submitting a PDF of the notebook output, or a link to the repository where the completed notebook is stored.

# Part 1 - Querying data using SQL commands
Since you already know quite a bit about querying a SQL database, this is the easiest way to get data from a database.

The basic steps are 
1. *Connect to the database*. This may require some setup work depending on database type (SQLite, MS SQL, Teradata, etc).  Often times an intermediate piece of software called a database driver is required. This piece of software knows how to make a connection to the database server and send your SQL commands to the database and return the results to you. For SQLite, the driver is built-into Python and so there is no extra software to setup other than the standard Python libraries.
2. *Create a cursor* that can be used to send commands to the server. A database cursor is a pointer that moves around the database and points to the data you want to retrieve from the database.
3. *Execute a command*. Once the setup has been completed, we can finally execute a command. A command is the SQL statement you want to be sent to the database.  If the command returns data (as in a SELECT statement), then the cursor will point to the data that is to be returned (but keep in mind, only 1 row of data at a time).

The next cell shows the basics of creating a connection, establishing a cursor and executing a command to the database.

In [1]:
import sqlite3

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

# Next make a cursor that can be used to run a query on the new connection
cur = cnn.cursor()

# See what tables are available in the database
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(tables.fetchall())


# get the first 10 records from the sales table
result = cur.execute('SELECT * from sales LIMIT 10')

# Get the names of the columns from a sqlite database
names = list(map(lambda x: x[0], cur.description))
print(names)

# Print the first 10 records
for row in result:
    print(row)  

# It's good practice to ensure that we close our connection to the database when we are done using it
cnn.close()

[('sales',)]
['sale_id', 'Configuration', 'Customer Postcode', 'Store Postcode', 'Retail Price', 'Screen Size (Inches)', 'Battery Life (Hours)', 'RAM (GB)', 'Processor Speeds (GHz)', 'Integrated Wireless?', 'HD Size (GB)', 'Bundled Applications?', 'customer X', 'customer Y', 'store X', 'store Y', 'sale_date']
(0, 163, 'EC4V 5BH', 'SE1 2BN', 455.0, 15, 5, 1, 2.0, 'Yes', 80, 'Yes', 532041, 180995, 534057.0, 179682.0, '2008-01-01 00:01:00')
(1, 320, 'SW4 0JL', 'SW12 9HD', 545.0, 15, 6, 1, 2.0, 'No', 300, 'No', 529240, 175537, 528739.0, 173080.0, '2008-01-01 00:02:00')
(2, 23, 'EC3V 1LR', 'E2 0RY', 515.0, 15, 4, 1, 2.0, 'Yes', 300, 'Yes', 533095, 181047, 535652.0, 182961.0, '2008-01-01 00:04:00')
(3, 169, 'SW1P 3AU', 'SE1 2BN', 395.0, 15, 5, 1, 2.0, 'No', 40, 'Yes', 529902, 179641, 534057.0, 179682.0, '2008-01-01 00:04:00')
(4, 365, 'EC4V 4EG', 'SW1V 4QQ', 585.0, 15, 6, 2, 2.0, 'No', 120, 'Yes', 531684, 180948, 528924.0, 178440.0, '2008-01-01 00:06:00')
(5, 309, 'W1B 5PX', 'SW1V 4QQ', 555.

Notice that each row of the query returns the result in a tuple.  A tuple similar to a list - it's a collection of values.  If we want to access a particular item in the results - we have to do so with an index.  Let's start by getting some data.

### Fetching Data Methods Explained:
- **`fetchall()`**: This method retrieves all the rows from the query result and loads them into memory. Be cautious when using this with large datasets as it can consume a lot of memory.
- **`fetchone()`**: Retrieves the next row from the query result. Useful when you expect a single result or want to iterate through results one at a time.
- **`fetchmany(size)`**: Retrieves the next 'size' number of rows from the query result. This is useful when you want to fetch data in manageable chunks rather than all at once or one at a time.

In this example, we're using `fetchall()` to get the first 10 records from the 'sales' table. Remember that the 'LIMIT 10' in the SQL query also restricts the output to 10 rows.

We are also using `fetchmany(5)` this will limit the results to just the first 5 even though the query will have access to 10 rows of data.  If we run `fetchmany(5)` again, we would get the second 5 results.

In [2]:
# We have to reopen the database and create a new cursor to run queries on it
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()

# We can also use the fetchall() method to get all the results at once
results = cur.execute('SELECT * from sales LIMIT 10').fetchall()
print(results)
    
# We can also use the fetchmany() method to get a specific number of results
results = cur.execute('SELECT * from sales LIMIT 10').fetchmany(5)
print(results)

[(0, 163, 'EC4V 5BH', 'SE1 2BN', 455.0, 15, 5, 1, 2.0, 'Yes', 80, 'Yes', 532041, 180995, 534057.0, 179682.0, '2008-01-01 00:01:00'), (1, 320, 'SW4 0JL', 'SW12 9HD', 545.0, 15, 6, 1, 2.0, 'No', 300, 'No', 529240, 175537, 528739.0, 173080.0, '2008-01-01 00:02:00'), (2, 23, 'EC3V 1LR', 'E2 0RY', 515.0, 15, 4, 1, 2.0, 'Yes', 300, 'Yes', 533095, 181047, 535652.0, 182961.0, '2008-01-01 00:04:00'), (3, 169, 'SW1P 3AU', 'SE1 2BN', 395.0, 15, 5, 1, 2.0, 'No', 40, 'Yes', 529902, 179641, 534057.0, 179682.0, '2008-01-01 00:04:00'), (4, 365, 'EC4V 4EG', 'SW1V 4QQ', 585.0, 15, 6, 2, 2.0, 'No', 120, 'Yes', 531684, 180948, 528924.0, 178440.0, '2008-01-01 00:06:00'), (5, 309, 'W1B 5PX', 'SW1V 4QQ', 555.0, 15, 6, 1, 2.0, 'Yes', 120, 'Yes', 529207, 180969, 528924.0, 178440.0, '2008-01-01 00:12:00'), (6, 75, 'SE25 4LX', 'SE8 3JD', 465.0, 15, 4, 2, 2.0, 'No', 80, 'Yes', 534575, 168236, 537175.0, 177885.0, '2008-01-01 00:14:00'), (7, 346, 'N7 6AR', 'E2 0RY', 450.0, 15, 6, 2, 1.5, 'No', 40, 'No', 530461, 186

In [3]:
# Now lets get just a single result and print out the SalesID, and the Sale Date.
results = cur.execute('SELECT * from sales LIMIT 10').fetchone()
print(results)
# Print the SalesID and Sale Date.  The SalesID is the first column, and the Sale Date is the 17th column
print(f'SalesID: {results[0]}, Sale Date: {results[16]}')

(0, 163, 'EC4V 5BH', 'SE1 2BN', 455.0, 15, 5, 1, 2.0, 'Yes', 80, 'Yes', 532041, 180995, 534057.0, 179682.0, '2008-01-01 00:01:00')
SalesID: 0, Sale Date: 2008-01-01 00:01:00


### Understanding Database Cursors and Data Fetching

When you execute a SQL query, the database doesn't immediately return all the data. Instead, it returns a **cursor** object. Think of the cursor as a pointer that navigates through the result set returned by a query. The cursor starts before the first row of the result set, and you can think of it as being positioned "above" the first row initially.

#### Cursor Movements

- **`fetchone()`**: When you call this method, the cursor moves down one row and returns that row's data. If you call `fetchone()` again, it will move down another row and return that row's data, and so on.
  
- **`fetchmany(size)`**: Calling this moves the cursor down by `size` number of rows and returns those rows as a list. For example, if you call `fetchmany(3)`, the cursor moves down three rows and returns those three rows.

- **`fetchall()`**: This method moves the cursor to the end of the table, fetching all remaining rows and returning them as a list. Note that this could be memory-intensive for large result sets.

#### Example

Imagine your query returns a result set like this, where the arrow (`-->`) indicates the cursor's position:
```
    (Row 1)
--> (Row 2)
    (Row 3)
```

After calling `fetchmany(2)`:
```
    (Row 1)
    (Row 2)
--> (Row 3)
```

After calling `fetchall()`:
```
    (Row 1)
    (Row 2)
    (Row 3)
--> (End)
```


#### Important Note
- Once the cursor reaches the end, subsequent calls to `fetchone()` or `fetchmany()` will return an empty result. You would need to re-execute the query to reset the cursor position.

Remember, the cursor is not just a Python concept; it's a database feature. So, when you execute a query, the cursor is maintained on the database server, not just in your Python script.

## YOUR TURN - Queries
Now that we know which columns are available to us we can query the database let's run a few queries.
Answer the questions in the following cells.  Ensure that your code and the result are available before saving to PDF.

As a reference, here are the column names:
```
sale_id
Configuration
Customer Postcode
Store Postcode
Retail Price
Screen Size (Inches)
Battery Life (Hours)
RAM (GB)
Processor Speeds (GHz)
Integrated Wireless?
HD Size (GB)
Bundled Applications?
customer X
customer Y
store X
store Y
sale_date
```

In [4]:
# The following steps are provided as a reminder of how to execute the query

# Open the database connection
# Make a connection to the database
# Create the cursor for which to execute the query
# Execute the query
# Print the results
# HINT: Since we are only looking for a single value - fetchone() is a good method to use
# Close the connection

# Q1: What is the total revenue from sales in the sales table? (the answer should be 144373314.0)
db_file = 'laptopsales.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
result = cur.execute("SELECT SUM([Retail Price]) FROM SALES").fetchone()
print(f'Total Revenue: {result[0]}')
cnn.close()


Total Revenue: 144373314.0


In [5]:
# Q2: Sum the sales by Store Postcode include the postcode in the results
# HINT: Now we are looking for multiple values - fetchall() is a good method to use

db_file = 'laptopsales.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
results = cur.execute("SELECT [Store Postcode], SUM([Retail Price]) FROM SALES GROUP BY [Store Postcode]").fetchall()
for row in results:
    print(f'Store Postcode: {row[0]}    Total Revenue: {row[1]}')

cnn.close()

Store Postcode: CR7 8LE    Total Revenue: 3117845.0
Store Postcode: E2 0RY    Total Revenue: 15972845.0
Store Postcode: E7 8NW    Total Revenue: 1479286.0
Store Postcode: KT2 5AU    Total Revenue: 2261810.0
Store Postcode: N17 6QA    Total Revenue: 2480320.0
Store Postcode: N3 1DH    Total Revenue: 1185805.0
Store Postcode: NW5 2QH    Total Revenue: 16364055.0
Store Postcode: S1P 3AU    Total Revenue: 72716.0
Store Postcode: SE1 2BN    Total Revenue: 23220815.0
Store Postcode: SE8 3JD    Total Revenue: 8640605.0
Store Postcode: SW12 9HD    Total Revenue: 6611185.0
Store Postcode: SW18 1NN    Total Revenue: 7324210.0
Store Postcode: SW1P 3AU    Total Revenue: 24338073.0
Store Postcode: SW1V 4QQ    Total Revenue: 22618985.0
Store Postcode: W10 6HQ    Total Revenue: 6101710.0
Store Postcode: W4 3PH    Total Revenue: 2583049.0


### Using the data from the queries
In all the examples the data comes back from the query as a tuple.  This means that we are getting all the values in a sort of collection of variables.  We can access the data in tuple just like it were a list.

Now, 
1. Create a class (LaptopSale) that has a property for each of the following columns in the database
   - SaleId, Configuration, SaleDate, HasIntWireless, RetailPrice (you are welcome to rename these property names however you see fit)
2. Query the database to get the first 20 items in the database, 
3. Create an object from each one and put it in a list
4. Then loop through the list you created
   * Print the SaleId, Configuration, HasIntWireless, RetailPrice and SaleDate for each items in list

***HINT:*** Here's code you can use for the query statement.  Make sure to copy the `"`
```python
   "SELECT sale_id, Configuration,sale_date, 'Integrated Wireless?', 'Retail Price' FROM sales LIMIT 20"
```
as in
```python
   cur.execute("SELECT sale_id, Configuration,sale_date, 'Integrated Wireless?', 'Retail Price' FROM sales LIMIT 20")
```

In [6]:
# Define the LaptopSale class
# This class will be used to store the results of the query
class LaptopSale:
    sale_id = 0
    configuration = 0
    sale_date = None
    integrated_wireless = ""
    retail_price = 0.0

    def __init__(self, sale_id, configuration, sale_date, integrated_wireless, retail_price):
        self.sale_id = sale_id
        self.configuration = configuration
        self.sale_date = sale_date
        self.integrated_wireless = integrated_wireless
        self.retail_price = retail_price


db_file = 'laptopsales.db'
# Open the database connection
cnn = sqlite3.connect(db_file)
# Create the cursor for which to execute the query
cur = cnn.cursor()

#table schema:
#['sale_id', 'Configuration', 'Customer Postcode', 'Store Postcode', 'Retail Price', 'Screen Size (Inches)', 'Battery Life (Hours)', 'RAM (GB)', 'Processor Speeds (GHz)', 'Integrated Wireless?', 'HD Size (GB)', 'Bundled Applications?', 'customer X', 'customer Y', 'store X', 'store Y', 'sale_date']

# Execute the query
results = cur.execute("SELECT sale_id, Configuration, sale_date, [Integrated Wireless?], [Retail Price] FROM sales LIMIT 20")

# Create an empty list to store the results
laptop_sales = []

for row in results:
# Loop through the results and create a new LaptopSale object for each row
    laptop_sale = LaptopSale(row[0], row[1], row[2], row[3], row[4])
# Add the new LaptopSale object to the list
    laptop_sales.append(laptop_sale)


#    print(f'sale_id: {row[0]}    Configuration: {row[1]}    sale_date: {row[2]}    Integrated Wireless?: {row[3]}    Retail Price: {row[4]}')


# Close the connection
cnn.close()


# Loop through the list of objects
for sale in laptop_sales:
# Print the object's Sale Id, Configuration and Sale date
    print(f'Sale Id: {sale.sale_id}     |       Configuration: {sale.configuration}     |       Sale Date: {sale.sale_date}')

Sale Id: 0     |       Configuration: 163     |       Sale Date: 2008-01-01 00:01:00
Sale Id: 1     |       Configuration: 320     |       Sale Date: 2008-01-01 00:02:00
Sale Id: 2     |       Configuration: 23     |       Sale Date: 2008-01-01 00:04:00
Sale Id: 3     |       Configuration: 169     |       Sale Date: 2008-01-01 00:04:00
Sale Id: 4     |       Configuration: 365     |       Sale Date: 2008-01-01 00:06:00
Sale Id: 5     |       Configuration: 309     |       Sale Date: 2008-01-01 00:12:00
Sale Id: 6     |       Configuration: 75     |       Sale Date: 2008-01-01 00:14:00
Sale Id: 7     |       Configuration: 346     |       Sale Date: 2008-01-01 00:17:00
Sale Id: 8     |       Configuration: 70     |       Sale Date: 2008-01-01 00:24:00
Sale Id: 9     |       Configuration: 351     |       Sale Date: 2008-01-01 00:56:00
Sale Id: 10     |       Configuration: 209     |       Sale Date: 2008-01-01 01:00:00
Sale Id: 11     |       Configuration: 359     |       Sale Date: 2

# Part 2 - Using pandas to connect to the database
If we are just looking to grab some data from a database and use it for analytics especially, then the [pandas](https://pandas.pydata.org/) library is a great choice.

To get data from SQL using `pandas` all we need to do is create the connection and execute the query


In [7]:
# Create connection
import sqlite3
import pandas as pd

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

data = pd.read_sql_query("SELECT * FROM sales LIMIT 10", cnn)

# In Jupyter, the last line of a cell will automatically be printed out without having to call the print() function
data

,sale_id,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y,sale_date
0,0,163,EC4V 5BH,SE1 2BN,455.0,15,5,1,2.0,Yes,80,Yes,532041,180995,534057.0,179682.0,2008-01-01 00:01:00
1,1,320,SW4 0JL,SW12 9HD,545.0,15,6,1,2.0,No,300,No,529240,175537,528739.0,173080.0,2008-01-01 00:02:00
2,2,23,EC3V 1LR,E2 0RY,515.0,15,4,1,2.0,Yes,300,Yes,533095,181047,535652.0,182961.0,2008-01-01 00:04:00
3,3,169,SW1P 3AU,SE1 2BN,395.0,15,5,1,2.0,No,40,Yes,529902,179641,534057.0,179682.0,2008-01-01 00:04:00
4,4,365,EC4V 4EG,SW1V 4QQ,585.0,15,6,2,2.0,No,120,Yes,531684,180948,528924.0,178440.0,2008-01-01 00:06:00
5,5,309,W1B 5PX,SW1V 4QQ,555.0,15,6,1,2.0,Yes,120,Yes,529207,180969,528924.0,178440.0,2008-01-01 00:12:00
6,6,75,SE25 4LX,SE8 3JD,465.0,15,4,2,2.0,No,80,Yes,534575,168236,537175.0,177885.0,2008-01-01 00:14:00
7,7,346,N7 6AR,E2 0RY,450.0,15,6,2,1.5,No,40,No,530461,186176,535652.0,182961.0,2008-01-01 00:17:00
8,8,70,W3 7JT,SW18 1NN,455.0,15,4,2,2.0,Yes,120,No,520898,180071,525155.0,175180.0,2008-01-01 00:24:00
9,9,351,SW8 1RB,SW1V 4QQ,620.0,15,6,2,1.5,No,300,Yes,530298,177435,528924.0,178440.0,2008-01-01 00:56:00


Now that our data is in a dataframe we can do all the things we know how to do with pandas already - like aggregate, filter, group and slice

In [8]:
# Calculate the average battery life
mean_battery_life = data['Battery Life (Hours)'].mean()
print(f'Average Battery Life: {mean_battery_life} hours')

# Get all sales where the laptop has integrated wireless
wireless_sales = data[data['Integrated Wireless?'] == 'Yes']
print(f'wireless sales count: {len(wireless_sales)}')

# Price difference between items with and without integrated wireless
wireless = data[data['Integrated Wireless?'] == 'Yes']['Retail Price'].mean()
no_wireless = data[data['Integrated Wireless?'] == 'No']['Retail Price'].mean()
print(f'Price difference between items with and without integrated wireless: {wireless - no_wireless}')


Average Battery Life: 5.2 hours
wireless sales count: 4
Price difference between items with and without integrated wireless: -15.0


## YOUR TURN - Using pandas to query data source
Now that you have seen how to connect to a database using pandas, we are going to move to a new database with a few more tables.  You can read about the chinook example [here](https://www.sqlitetutorial.net/sqlite-sample-database/).

![chinook database](images/sqlite-sample-database-color.jpg)

Complete the following queries and show your results

In [9]:
# Create connection
db_file = 'chinook.db'
cnn = sqlite3.connect(db_file)

# Write a query that will return the artist name, album title, and genre name into a dataframe
data = pd.read_sql_query(
    """SELECT DISTINCT a.Name AS artist_name
        , alb.Title AS album_title
        , g.Name AS genre_name
    FROM artists a
    JOIN albums alb
        ON a.ArtistId = alb.ArtistId
    JOIN tracks t
        ON t.AlbumId = alb.AlbumId
    JOIN genres g
        ON g.GenreId = t.GenreId"""
    , cnn
)
# Close the connection
cnn.close()

# Show the dataframe
data

,artist_name,album_title,genre_name
0,AC/DC,For Those About To Rock We Salute You,Rock
1,Accept,Balls to the Wall,Rock
2,Accept,Restless and Wild,Rock
3,AC/DC,Let There Be Rock,Rock
4,Aerosmith,Big Ones,Rock
...,...,...,...
355,Eugene Ormandy,Respighi:Pines of Rome,Classical
356,Emerson String Quartet,Schubert: The Late String Quartets & String Qu...,Classical
357,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,Classical
358,Nash Ensemble,Mozart: Chamber Music,Classical


### Dynamic Query
For this next exercise you need to ask the user for a band name and then use this name to get all the albums for that band into a dataframe.

***HINTS:***
1. Consider building the query with string manipulation
2. It might be difficult to get the band name spelled exactly right, so use the SQL operator `LIKE` to help with getting close
3. To get user input you will use the `input()` function
4. Review [this lesson](https://www.py4analytics.info/book/lessons/250-Database_Connections.html#dynamic-queries) on dynamic queries if you need some additional help

In [10]:
artist_name = input("What band are you interested in?  Put a partial name if you don't know the full name: ")

db_file = 'chinook.db'
cnn = sqlite3.connect(db_file)

#This is what I was able to figure out in order to have it allow the LIKE clause to work correctly, but it feels wrong. Is there a better way?
#I considered placing the artist_name variable directly in the SQL, but I think that could allow SQL injection.
artist_name = "%" + artist_name + "%" 
params = (artist_name,)

# Your code for the dynamic query goes here
query =  """SELECT DISTINCT a.Name AS artist_name
        , alb.Title AS album_title
        , g.Name AS genre_name
    FROM artists a
    JOIN albums alb
        ON a.ArtistId = alb.ArtistId
    JOIN tracks t
        ON t.AlbumId = alb.AlbumId
    JOIN genres g
        ON g.GenreId = t.GenreId
    WHERE a.Name LIKE ?  """
data = pd.read_sql_query(query, cnn, params=params)
cnn.close()
data


,artist_name,album_title,genre_name
0,AC/DC,For Those About To Rock We Salute You,Rock
1,Accept,Balls to the Wall,Rock
2,Accept,Restless and Wild,Rock
3,AC/DC,Let There Be Rock,Rock
4,BackBeat,BackBeat Soundtrack,Rock And Roll
5,Black Label Society,Alcohol Fueled Brewtality Live! [Disc 1],Metal
6,Black Label Society,Alcohol Fueled Brewtality Live! [Disc 2],Metal
7,Black Sabbath,Black Sabbath,Metal
8,Black Sabbath,Black Sabbath Vol. 4 (Remaster),Metal
9,Godsmack,Faceless,Metal


# Part 3 - Using an SQLAlchemy (Object Relational Mapping)
So far you have learned how to work with databases by querying directly, using dynamic queries and getting the data back in the form of a dataframe.  What would be super handy in some cases is to be able to map database objects directly onto Python objects. Being able to treat database tables like other Python objects is very valuable.  Expecting each row to be represented by a particular Python object allows us to access the columns as properties of the class and handles the insert/update/delete functions for us without having to depend on fragile SQL strings.  (Fragile SQL strings means that we are putting code logic into a string which isn't checked by Python before being run, so if there is a change in the database, we have to search through strings in our code to update the logic rather than being able to depend on refactoring tools provided by our IDEs.  *Are you glad you asked what 'fragile SQL strings' means?*)  The library most often used to do ORM (object relational mapping) is a library called SQLAlchemy.

Going back to our `chinook` dataset.  It would be helpful to be able to consider customers as Python objects, if we could query for them and then use the objects to build our front-end forms or maybe just export them to a semi-strucured file format like json or xml for transport.  We'll start with an example and then break it down.

**NOTE: If you haven't installed [SQLAlchemy](https://www.sqlalchemy.org/) yet you are going to need to do so.  You can reference this site for the [installation guide](https://docs.sqlalchemy.org/en/20/intro.html#installation).  Try not to get to mired in the details of the rest of the library just yet**

In [11]:
import sqlite3
import sqlalchemy
# install SQLAlchemy using pip install SQLAlchemy
from sqlalchemy import create_engine, text

if sqlalchemy.__version__ < '2.0.0':
    raise ValueError('Please upgrade your version of SQLAlchemy to 2.0.0 or greater')

engine = create_engine('sqlite:///chinook.db')
with engine.connect() as conn:
    # Get the top 10 albums
    results = conn.execute(text("SELECT * FROM albums LIMIT 10")).fetchall()

for album in results:
    print(album)

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)


So far we see we can use SQLAlchemy in the same way we used connections to the SQLite3 database already.  Let's take it one step further.

**NOTE: It may be helpful to review the [ORM Quick Start](https://docs.sqlalchemy.org/en/20/orm/quickstart.html) provided by SQLAlchemy in conjunction with the following**

**NOTE2: If you are not using the UofA Jupyter environment you may need to install SQLAlchemy.  See the [installation guide](https://docs.sqlalchemy.org/en/20/intro.html#installation)**

In [12]:
from typing import List, Optional 
from sqlalchemy import Integer, String, DateTime, Float, ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Session, relationship
from sqlalchemy import select

class Base(DeclarativeBase):
    pass

# Notice we are naming the class in the singular, even though the table name is plural
# This is because each instance of the class represents a single row in the table
class Invoice(Base):
    __tablename__ = 'invoices'
    
    id: Mapped[int]= mapped_column('InvoiceId', primary_key=True)
    invoiceDate = mapped_column(DateTime)
    billingAddress = mapped_column(String)
    billingCity= mapped_column(String)
    billingState= mapped_column(String)
    billingCountry = mapped_column(String)
    billingPostalCode = mapped_column(String)
    total= mapped_column(Float)

    # Create a list to hold the invoice details associated with this invoice
    invoice_items: Mapped[List['Invoice_Item']] = relationship(back_populates='invoice')

    # This is a special method that is called when we print an instance of the class
    def __repr__(self):
        return f'Invoice(id={self.id}, invoiceDate={self.invoiceDate}, billingAddress={self.billingAddress}, billingCity={self.billingCity}, billingState={self.billingState}, billingCountry={self.billingCountry}, billingPostalCode={self.billingPostalCode}, total={self.total})'
    

# A class to represent the invoice items table
class Invoice_Item(Base):
    __tablename__ = 'invoice_items'
    
    id: Mapped[int]= mapped_column('InvoiceLineId', primary_key=True)    
    # This provides us a link back to the invoice table
    # Notice we are using the more Python appropriate invoice_id instead of InvoiceId which is the column name in the table
    invoice_id : Mapped[int]= mapped_column('InvoiceId', ForeignKey('invoices.InvoiceId'))
    invoice : Mapped[Invoice]= relationship('Invoice', back_populates='invoice_items')
    
    track_id : Mapped[int]= mapped_column('trackId', ForeignKey('tracks.TrackId'))
    track: Mapped['Track']= relationship('Track')
    
    unit_price : Mapped[float]= mapped_column('unitPrice')
    quantity : Mapped[int]= mapped_column('quantity')

# Let's add a few more classes so we can get all the details of the orders from the database
class Track(Base):
    __tablename__ = 'tracks'
    
    id: Mapped[int]= mapped_column('TrackId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')
    
    # Define the album id column and link it to the album table
    album_id: Mapped[int]= mapped_column('AlbumId', ForeignKey('albums.AlbumId'))
    album: Mapped['Album']= relationship('Album')
    
    # Define the media type id column and link it to the media type table
    media_type_id: Mapped[int]= mapped_column('MediaTypeId', ForeignKey('media_types.MediaTypeId'))
    media_type : Mapped['MediaType']= relationship('MediaType')
    
    # Define the genre id column and link it to the genre table
    genre_id: Mapped[int]= mapped_column('GenreId', ForeignKey('genres.GenreId'))
    genre : Mapped['Genre']= relationship('Genre')
    
    composer: Mapped[str]= mapped_column('Composer')
    milliseconds: Mapped[int]= mapped_column('Milliseconds')
    bytes: Mapped[int]= mapped_column('Bytes')
    unit_price: Mapped[float]= mapped_column('UnitPrice')
    
# Class that maps to MediaType
class MediaType(Base):
    __tablename__ = 'media_types'
    
    id: Mapped[int]= mapped_column('MediaTypeId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

# Class that maps to Genre
class Genre(Base):
    __tablename__ = 'genres'
    
    id: Mapped[int]= mapped_column('GenreId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

# Class that maps to Album
class Album(Base):
    __tablename__ = 'albums'
    
    id: Mapped[int]= mapped_column('AlbumId', primary_key=True)
    title: Mapped[str]= mapped_column('Title')

    artist_id: Mapped[int]= mapped_column('ArtistId', ForeignKey('artists.ArtistId'))
    artist: Mapped['Artist']= relationship('Artist')

# Class that maps to Artist
class Artist(Base):
    __tablename__ = 'artists'
    
    id: Mapped[int]= mapped_column('ArtistId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

session = Session(engine)
result = session.query(Invoice).limit(10)
for invoice in result:
    print(invoice)

# We want to ensure that we are always closing the session after we are done using it
session.close()

Invoice(id=1, invoiceDate=2009-01-01 00:00:00, billingAddress=Theodor-Heuss-Straße 34, billingCity=Stuttgart, billingState=None, billingCountry=Germany, billingPostalCode=70174, total=1.98)
Invoice(id=2, invoiceDate=2009-01-02 00:00:00, billingAddress=Ullevålsveien 14, billingCity=Oslo, billingState=None, billingCountry=Norway, billingPostalCode=0171, total=3.96)
Invoice(id=3, invoiceDate=2009-01-03 00:00:00, billingAddress=Grétrystraat 63, billingCity=Brussels, billingState=None, billingCountry=Belgium, billingPostalCode=1000, total=5.94)
Invoice(id=4, invoiceDate=2009-01-06 00:00:00, billingAddress=8210 111 ST NW, billingCity=Edmonton, billingState=AB, billingCountry=Canada, billingPostalCode=T6G 2C7, total=8.91)
Invoice(id=5, invoiceDate=2009-01-11 00:00:00, billingAddress=69 Salem Street, billingCity=Boston, billingState=MA, billingCountry=USA, billingPostalCode=2113, total=13.86)
Invoice(id=6, invoiceDate=2009-01-19 00:00:00, billingAddress=Berger Straße 10, billingCity=Frankfurt,

What's going on here. In the cell above, we create a class called ```Invoice``` which maps directly to the ```Invoice``` table in the database.  SQLAlchemy knows how to convert *objects* of type ```Invoice``` into rows in the database.  This saves us the effort and fragility of updating custom sql statements.

What's more, we can establish the relationship between ```invoice_items``` and ```invoices```, meaning we can easy find the ```invoice``` related to an ```invoice_items``` and likewise, we have a list of ```invoice_items``` represented on an ```invoice```.

Furthermore, we can leverage Python to do many of the same things we could do in SQL using SQL like functions.  Take a look at the next cell

### Adding this section because it bugs me to not be able to see what tables/columns are in the database

In [13]:
db_file = 'chinook.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

table_list = []
for table in tables:
    table_list.append(table)

for table_name in table_list:
    result = cur.execute(f'SELECT * from {table_name[0]} LIMIT 10;')
    names = list(map(lambda x: x[0], cur.description))
    print("Table Name: " + table_name[0])
    print("Columns: " + str(names))

cnn.close()

Table Name: albums
Columns: ['AlbumId', 'Title', 'ArtistId']
Table Name: sqlite_sequence
Columns: ['name', 'seq']
Table Name: artists
Columns: ['ArtistId', 'Name']
Table Name: customers
Columns: ['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
Table Name: employees
Columns: ['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']
Table Name: genres
Columns: ['GenreId', 'Name']
Table Name: invoices
Columns: ['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode', 'Total']
Table Name: invoice_items
Columns: ['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']
Table Name: media_types
Columns: ['MediaTypeId', 'Name']
Table Name: playlists
Columns: ['PlaylistId', 'Name']
Table Name: playlist_track
Column

#### Data Dictionary

In [14]:
db_file = 'chinook.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

def bool_to_yes_no(value):
    if value == 0:
        return 'NO'
    elif value == 1:
        return 'YES'

for table in tables:
    print('------------------')
    print('Table Name: ' + table[0])
    print('------------------')
    columns = cur.execute(f'PRAGMA table_info(\'%s\');' % table[0]).fetchall()
    for column in columns:
        col_id, col_name, col_type, col_notnull, col_default, col_pk = column
        print(f"  Column: {col_name}")
        print(f"    Type: {col_type}")
        print(f"    Not Null: {bool_to_yes_no(col_notnull)}")
        print(f"    Default Value: {col_default}")
        print(f"    Primary Key: {bool_to_yes_no(col_pk)}")

cnn.close()

------------------
Table Name: albums
------------------
  Column: AlbumId
    Type: INTEGER
    Not Null: YES
    Default Value: None
    Primary Key: YES
  Column: Title
    Type: NVARCHAR(160)
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: ArtistId
    Type: INTEGER
    Not Null: YES
    Default Value: None
    Primary Key: NO
------------------
Table Name: sqlite_sequence
------------------
  Column: name
    Type: 
    Not Null: NO
    Default Value: None
    Primary Key: NO
  Column: seq
    Type: 
    Not Null: NO
    Default Value: None
    Primary Key: NO
------------------
Table Name: artists
------------------
  Column: ArtistId
    Type: INTEGER
    Not Null: YES
    Default Value: None
    Primary Key: YES
  Column: Name
    Type: NVARCHAR(120)
    Not Null: NO
    Default Value: None
    Primary Key: NO
------------------
Table Name: customers
------------------
  Column: CustomerId
    Type: INTEGER
    Not Null: YES
    Default Value: None
    P

In [15]:
# Create a new session
session = Session(engine)

# Get the first invoice using the id
invoice_1 = session.query(Invoice).filter(Invoice.id == 1).first()
print (f'Invoice 1: {invoice_1.invoiceDate}')

# Get the first 10 invoices, using something more like SQL
statement = select(Invoice).where(Invoice.id <= 10)
for invoice in session.scalars(statement):
    print(f'Invoice {invoice.id}: {invoice.invoiceDate}')

# Close the session
session.close()


Invoice 1: 2009-01-01 00:00:00
Invoice 1: 2009-01-01 00:00:00
Invoice 2: 2009-01-02 00:00:00
Invoice 3: 2009-01-03 00:00:00
Invoice 4: 2009-01-06 00:00:00
Invoice 5: 2009-01-11 00:00:00
Invoice 6: 2009-01-19 00:00:00
Invoice 7: 2009-02-01 00:00:00
Invoice 8: 2009-02-01 00:00:00
Invoice 9: 2009-02-02 00:00:00
Invoice 10: 2009-02-03 00:00:00


The real value of the ORM (object relational model) mode of SQLAlchemy is when there are relationships between items in the database.  For instance, if we look closer at the `invoice` and `invoice_items` tables we see they are related.  It would be super handy to be able to load up our invoice_items when we load up a specific invoice.  

<p align="center"><img src="./images/a3-invoices.png" alt="close-up of the invoices and invoice_items tables"/>
</p>
<span style="color:red">***BEWARE: The image shows that the key to the ```invoice_items``` table is called ```InvoiceItemId``` in reality it is called ```InvoiceLineId```.***</span>

In [16]:
# Create a new session
session = Session(engine)

# Get the first invoice using the id
invoice_1 = session.query(Invoice).filter(Invoice.id == 1).first()
print (f'Invoice 1: {invoice_1.invoiceDate}')
    
invoice_items = invoice_1.invoice_items
print(f'Invoice 1 has {len(invoice_items)} invoice items')  
for invoice_item in invoice_items:
    print(f'Invoice Item {invoice_item.id}: {invoice_item.unit_price}, {invoice_item.quantity}')

# Close the session to ensure that we are not keeping a connection open
session.close()

Invoice 1: 2009-01-01 00:00:00
Invoice 1 has 2 invoice items
Invoice Item 1: 0.99, 1
Invoice Item 2: 0.99, 1


So just by querying the invoice we also have access to the details of the invoice. Since we are interested in all the details there is no need to create complex join statements like we would before to get the information we needed.  

Now, let's add the `Track` information to round out the picture.  We are going to skip `playlists` and `customers` because they introduce some additional complexity which isn't as easy to map, but which definitely can be done.  The next cell demonstrates the pattern we've seen earlier where we define the *id* field as a foreign key and then immediately follow this with a property which holds (one or more) of the objects represented by the row in the parent table.  For instance,

```python
    media_type_id: Mapped[int]= mapped_column('MediaTypeId', ForeignKey('media_types.MediaTypeId'))
    media_type : Mapped['MediaType']= relationship('MediaType')
```

SQLAlchemy will get the media_type_id for us, but also, it is smart enough to load an object of type `MediaType` for us.  Allowing us to get the useful part, the name of the media_type rather quickly
```python
    track.media_type.name
```
*While the order of the property definitions doesn't matter, the code is easier to read if these two lines are kept together.*

In [17]:
# Let's look for the artist name and album for the first 10 albums in the database
session = Session(engine)
statement = select(Album).limit(10)
for album in session.scalars(statement):
    print(f'Album: {album.title}, Artist: {album.artist.name}')
session.close()

Album: For Those About To Rock We Salute You, Artist: AC/DC
Album: Balls to the Wall, Artist: Accept
Album: Restless and Wild, Artist: Accept
Album: Let There Be Rock, Artist: AC/DC
Album: Big Ones, Artist: Aerosmith
Album: Jagged Little Pill, Artist: Alanis Morissette
Album: Facelift, Artist: Alice In Chains
Album: Warner 25 Anos, Artist: Antônio Carlos Jobim
Album: Plays Metallica By Four Cellos, Artist: Apocalyptica
Album: Audioslave, Artist: Audioslave


## YOUR TURN - Querying data using SQLAlchemy
Now we are going to use a bit of what we know to get some information from the database using SQLAlchemy.  If you haven't done so a quick review of the [ORM Quickstart](https://docs.sqlalchemy.org/en/20/orm/quickstart.html) for SQLAlchemy may be helpful.  Also, you may find the [ORM Querying Guide](https://docs.sqlalchemy.org/en/20/orm/queryguide/index.html) helpful.

- Always start by writing in English (in comments) what you want to complete prior to writing code.
- Refer to the previous cells for examples


In [18]:
# Get the track name for the first 10 tracks in the database

# Create the session
session = Session(engine)

# Query the tracks
statement = select(Track).limit(10)

# For each track, print the track name
for track in session.scalars(statement):
    print(f'Track: {track.name}')

# Close the session
session.close()


Track: For Those About To Rock (We Salute You)
Track: Balls to the Wall
Track: Fast As a Shark
Track: Restless and Wild
Track: Princess of the Dawn
Track: Put The Finger On You
Track: Let's Get It Up
Track: Inject The Venom
Track: Snowballed
Track: Evil Walks


In [19]:

# Get the track name and genre name for the first 10 tracks in the database

# Create the session
session = Session(engine)

# Query the tracks
statement = select(Track).limit(10)

# For each track, print the track name and genre name

# I know I went a little off the rails for this section, but I was basically done with everything and wanted to test different ways to make the output nicer. I figured
# it'll also be good to know how to format the output for the group project.
print('|' + (' ' * 23) + 'Track' + (' ' * 24) + '|' + (' ' * 8) + 'Genre' + (' ' * 8) + '|')
print('-' * 76)
for track in session.scalars(statement):
# I'm adding spacing to get everything to align, please don't judge me:
    padded_track_name = track.name + ' ' * (50 - len(track.name))
    padded_genre_name = track.genre.name + ' ' * (20 - len(track.genre.name)) + '|'
    print(f'| {padded_track_name} | {padded_genre_name}')

# Close the session
session.close()

|                       Track                        |        Genre        |
----------------------------------------------------------------------------
| For Those About To Rock (We Salute You)            | Rock                |
| Balls to the Wall                                  | Rock                |
| Fast As a Shark                                    | Rock                |
| Restless and Wild                                  | Rock                |
| Princess of the Dawn                               | Rock                |
| Put The Finger On You                              | Rock                |
| Let's Get It Up                                    | Rock                |
| Inject The Venom                                   | Rock                |
| Snowballed                                         | Rock                |
| Evil Walks                                         | Rock                |
